In [1]:
# install speech recognition
!pip install SpeechRecognition

# install pyaudio
!pip install pyaudio

#install pyttsx3
!pip install pyttsx3

#install torch
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.16.1%2Bcu118-cp310-cp310-win_amd64.whl (4.9 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.1.1%2Bcu118-cp310-cp310-win_amd64.whl (3.9 MB)


In [2]:
#set up hugging face
!pip install transformers


In [1]:
!pip install tensorflow

  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic-core 2.14.5 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.


In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
tokenizer = GPT2Tokenizer.from_pretrained("af1tang/personaGPT")
model = GPT2LMHeadModel.from_pretrained("af1tang/personaGPT")
if torch.cuda.is_available():
    model = model.cuda()
## utility functions ##
flatten = lambda l: [item for sublist in l for item in sublist]

def to_data(x):
    if torch.cuda.is_available():
        x = x.cpu()
    return x.data.numpy()

def to_var(x):
    if not torch.is_tensor(x):
        x = torch.Tensor(x)
    if torch.cuda.is_available():
        x = x.cuda()
    return x

def display_dialog_history(dialog_hx):
    for j, line in enumerate(dialog_hx):
        msg = tokenizer.decode(line)
        if j %2 == 0:
            print(">> User: "+ msg)
        else:
            print("Bot: "+msg)
            print()

def generate_next(bot_input_ids, do_sample=True, top_k=10, top_p=.92,
                  max_length=1000, pad_token=tokenizer.eos_token_id):
    full_msg = model.generate(bot_input_ids, do_sample=True,
                                              top_k=top_k, top_p=top_p, 
                                              max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    msg = to_data(full_msg.detach()[0])[bot_input_ids.shape[-1]:]
    return msg


In [4]:
# get personality facts for conversation
personas = []
for i in range(3):
    response = input(">> Fact %d: "%(i+1))+ tokenizer.eos_token
    personas.append(response)
personas = tokenizer.encode(''.join(['<|p2|>'] + personas + ['<|sep|>'] + ['<|start|>']))


In [5]:
# converse for 8 turns
dialog_hx = []
for step in range(8):
    # encode the user input
    user_inp = tokenizer.encode(input(">> User: ") + tokenizer.eos_token)
    # append to the chat history
    dialog_hx.append(user_inp)
        
    # generated a response while limiting the total chat history to 1000 tokens, 
    bot_input_ids = to_var([personas + flatten(dialog_hx)]).long()
    msg = generate_next(bot_input_ids)
    dialog_hx.append(msg)
    print("Bot: {}".format(tokenizer.decode(msg, skip_special_tokens=True)))

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Bot: i am good. just playing with my dog.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Bot: i don't know. do you have a dog?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

Bot: they're good dogs


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Bot: they're nice dogs


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Bot: why not?


In [7]:
import pyttsx3

def speak_text(text):

    # Initialize the TTS engine
    engine = pyttsx3.init()

    # Set the rate of speech (optional)
    rate = engine.getProperty('rate')
    engine.setProperty('rate', rate - 50)

    # Set the voice (optional)
    # You can check available voices using engine.getProperty('voices')
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)

    # Speak the text
    engine.say(text)

    # Wait for the speech to finish
    engine.runAndWait()


In [8]:
import speech_recognition as sr
import pyaudio
# recognizer class
r = sr.Recognizer()

# listen to audio

# process audio okish
while True:
    with sr.Microphone() as source:
        print("listening")
        audio = r.listen(source,5, phrase_time_limit=5)
    try:
        r.recognize_google(audio)
        text = r.recognize_google(audio)
        print("you speak: ", text) # send text to processing
        speak_text(text)
    except sr.UnknownValueError:
        response = "What are you talking about"
        speak_text(response)
    except sr.RequestError as e:
        response = "What are you talking about"
        speak_text(response)
    

listening
listening
you speak:  maybe you have to I don't think it's able to listen to us right now yeah
listening
you speak:  I think it's kind of like because I said the timer
listening


KeyboardInterrupt: 